# COMPUTER VISION AND PATTERN RECOGNITION PROJECT
## Alessandro Cesa

In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import copy

In [2]:
!wget -c https://github.com/AlessandroCesaTs/Computer_Vision_Project_Cesa/archive/main.zip
!unzip main.zip

--2023-12-23 18:06:16--  https://github.com/AlessandroCesaTs/Computer_Vision_Project_Cesa/archive/main.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/AlessandroCesaTs/Computer_Vision_Project_Cesa/zip/refs/heads/main [following]
--2023-12-23 18:06:17--  https://codeload.github.com/AlessandroCesaTs/Computer_Vision_Project_Cesa/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [        <=>         ] 164.52M  29.9MB/s    in 5.6s    

2023-12-23 18:06:26 (29.4 MB/s) - ‘main.zip’ saved [172509724]

Archive:  main.zip
aff02ff9b837cfb9fae08a9e70fbc572b5f35b0f
   creating

In [3]:
train_path = "Computer_Vision_Project_Cesa-main/CVPR2023_project_2_and_3_data/train"
test_path = "Computer_Vision_Project_Cesa-main/CVPR2023_project_2_and_3_data/test"

In [4]:
transform=transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor(),transforms.Grayscale()])

In [5]:
train=ImageFolder(root=train_path,transform=transform)
test=ImageFolder(root=test_path,transform=transform)

In [6]:
"""
# check the dataset size
print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")
print(train.classes)
"""

'\n# check the dataset size\nprint(f"Train size: {len(train)}")\nprint(f"Test size: {len(test)}")\nprint(train.classes)\n'

In [7]:
"""
# get image no. 1 and its label
index=1
img,label=train[index]

print(f"label: {label}")

print("class: " + train.classes[label])
# display
figure = plt.figure()
plt.imshow(img.numpy().transpose((1, 2, 0)),cmap='gray') #<< swap axes because img is (3,H,W) but imshow() expects (H,W,3)
"""

'\n# get image no. 1 and its label\nindex=1\nimg,label=train[index]\n\nprint(f"label: {label}")\n\nprint("class: " + train.classes[label])\n# display\nfigure = plt.figure()\nplt.imshow(img.numpy().transpose((1, 2, 0)),cmap=\'gray\') #<< swap axes because img is (3,H,W) but imshow() expects (H,W,3)\n'

In [8]:
#split training set into training and validation
train_size=int(0.85*len(train))
validation_size=len(train)-train_size
training_set,validation_set=torch.utils.data.random_split(train,[train_size,validation_size])

In [9]:
# Create data loaders.
# Data loaders are basically objects that make easy to iterate through the data by batches of some batch_size

batch_size = 32
train_loader = DataLoader(training_set, batch_size=batch_size,shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=batch_size,shuffle=False)
test_loader = DataLoader(test, batch_size=batch_size,shuffle=False)

for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

"""
# get a batch from the training set
dataiter = iter(train_loader)
images, labels = next(dataiter)   #images is a (batch_size,3,32,32) tensor

print(images.size())
"""

Shape of X [N, C, H, W]: torch.Size([32, 1, 64, 64])
Shape of y: torch.Size([32]) torch.int64


'\n# get a batch from the training set\ndataiter = iter(train_loader)\nimages, labels = next(dataiter)   #images is a (batch_size,3,32,32) tensor\n\nprint(images.size())\n'

In [10]:
"""
classes=train.classes

# get a batch from the training set
dataiter = iter(train_loader)
images, labels = next(dataiter)   #images is a (batch_size,3,32,32) tensor

print(images.size())

img=torchvision.utils.make_grid(images)
figure = plt.figure()
plt.imshow(img.numpy().transpose((1, 2, 0)),cmap='gray') #<< swap axes because img is (3,H,W) but imshow() expects (H,W,3)

print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
"""

"\nclasses=train.classes\n\n# get a batch from the training set\ndataiter = iter(train_loader)\nimages, labels = next(dataiter)   #images is a (batch_size,3,32,32) tensor\n\nprint(images.size())\n\nimg=torchvision.utils.make_grid(images)\nfigure = plt.figure()\nplt.imshow(img.numpy().transpose((1, 2, 0)),cmap='gray') #<< swap axes because img is (3,H,W) but imshow() expects (H,W,3)\n\nprint(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))\n"

In [11]:
"""
# display the last image only
figure = plt.figure()
plt.imshow(images[-1,:,:,:].numpy().transpose((1,2,0)),cmap='gray')
print(f'{train.classes[labels[-1]]:5s}')
"""

"\n# display the last image only\nfigure = plt.figure()\nplt.imshow(images[-1,:,:,:].numpy().transpose((1,2,0)),cmap='gray')\nprint(f'{train.classes[labels[-1]]:5s}')\n"

In [12]:
#build the network

class simpleCNN(nn.Module):
  def __init__(self):
    super(simpleCNN,self).__init__() #initialize the model

    self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,stride=1) #Output image size is (size+2*padding-kernel)/stride -->62*62
    self.relu1=nn.ReLU()
    self.maxpool1=nn.MaxPool2d(kernel_size=2,stride=2) #outtput image 62/2-->31*31

    self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1) #output image is 29*29
    self.relu2=nn.ReLU()
    self.maxpool2=nn.MaxPool2d(kernel_size=2,stride=2) #output image is 29/2-->14*14  (MaxPool2d approximates size with floor)

    self.conv3=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1) #output image is 12*12
    self.relu3=nn.ReLU()

    self.fc1=nn.Linear(32*12*12,15) #16 channels * 16*16 image (64*64 with 2 maxpooling of stride 2), 15 output features=15 classes
    self.softmax = nn.Softmax()
    self.output=nn.Linear(15,15)

  def forward(self,x):
    x=self.conv1(x)
    x=self.relu1(x)
    x=self.maxpool1(x)

    x=self.conv2(x)
    x=self.relu2(x)
    x=self.maxpool2(x)

    x=self.conv3(x)
    x=self.relu3(x)

    x=x.view(-1,32*12*12)

    x=self.fc1(x)
    x=self.softmax(x)
    x=self.output(x)

    return x



In [13]:
def init_weights(m):
  if isinstance(m,nn.Conv2d) or isinstance(m,nn.Linear):
    nn.init.normal_(m.weight,0,0.01)
    nn.init.zeros_(m.bias)

In [14]:

# Instantiate the model
model = simpleCNN()

model.apply(init_weights)
# Print the model summary
print(model)


simpleCNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (fc1): Linear(in_features=4608, out_features=15, bias=True)
  (softmax): Softmax(dim=None)
  (output): Linear(in_features=15, out_features=15, bias=True)
)


In [15]:
#Training
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
#optimizer = optim.Adam(model.parameters())

def train_one_epoch(epoch_index,loader):
  running_loss=0

  for i, data in enumerate(loader):

    inputs,labels=data #get the minibatch
    outputs=model(inputs) #forward pass

    loss=loss_function(outputs,labels) #compute loss
    running_loss+=loss.item() #sum up the loss for the minibatches processed so far

    optimizer.zero_grad() #reset gradients
    loss.backward() #compute gradient
    optimizer.step() #update weights

  return running_loss/(i+1) # average loss per minibatch


In [16]:

EPOCHS=20

best_validation_loss=np.inf

for epoch in range(EPOCHS):
  print('EPOCH{}:'.format(epoch+1))

  model.train(True)
  train_loss=train_one_epoch(epoch,train_loader)

  running_validation_loss=0.0

  model.eval()

  with torch.no_grad(): # Disable gradient computation and reduce memory consumption
    for i,vdata in enumerate(validation_loader):
      vinputs,vlabels=vdata
      voutputs=model(vinputs)
      vloss=loss_function(voutputs,vlabels)
      running_validation_loss+=vloss
  validation_loss=running_validation_loss/(i+1)
  print('LOSS train: {} validation: {}'.format(train_loss,validation_loss))

  if validation_loss<best_validation_loss: #save the model if it's the best so far
    timestamp=datetime.now().strftime('%Y%m%d_%H%M%S')
    best_validation_loss=validation_loss
    model_path='model_{}_{}'.format(timestamp,epoch)
    torch.save(model.state_dict(),model_path)


EPOCH1:


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


LOSS train: 2.7081305921077727 validation: 2.708916664123535
EPOCH2:
LOSS train: 2.708078110218048 validation: 2.7093796730041504
EPOCH3:
LOSS train: 2.7080434560775757 validation: 2.709463357925415
EPOCH4:
LOSS train: 2.7079479694366455 validation: 2.7099199295043945
EPOCH5:
LOSS train: 2.7079000413417815 validation: 2.710268020629883
EPOCH6:
LOSS train: 2.7078364312648775 validation: 2.7106473445892334
EPOCH7:
LOSS train: 2.707805705070496 validation: 2.710867404937744
EPOCH8:
LOSS train: 2.707729196548462 validation: 2.7111523151397705
EPOCH9:
LOSS train: 2.707713556289673 validation: 2.711433172225952
EPOCH10:
LOSS train: 2.707686352729797 validation: 2.711876153945923
EPOCH11:
LOSS train: 2.7076858401298525 validation: 2.712128162384033
EPOCH12:
LOSS train: 2.707556241750717 validation: 2.7123777866363525
EPOCH13:
LOSS train: 2.707548898458481 validation: 2.712580680847168
EPOCH14:
LOSS train: 2.707548987865448 validation: 2.7129592895507812
EPOCH15:
LOSS train: 2.7074772834777834

In [17]:

#load the best model and evaluate performance on the test set

newModel=simpleCNN()
newModel.load_state_dict(torch.load(model_path))

correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=newModel(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy of the network on the test images: {100*correct/total}%")


Accuracy of the network on the test images: 5.360134003350084%


In [18]:
model = simpleCNN()
#Training
loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
#optimizer = optim.Adam(model.parameters())

def train_one_epoch(epoch_index,loader):
  running_loss=0

  for i, data in enumerate(loader):

    inputs,labels=data #get the minibatch
    outputs=model(inputs) #forward pass

    loss=loss_function(outputs,labels) #compute loss
    running_loss+=loss.item() #sum up the loss for the minibatches processed so far

    optimizer.zero_grad() #reset gradients
    loss.backward() #compute gradient
    optimizer.step() #update weights

  return running_loss/(i+1) # average loss per minibatch
EPOCHS=20

best_validation_loss=np.inf

for epoch in range(EPOCHS):
  print('EPOCH{}:'.format(epoch+1))

  model.train(True)
  train_loss=train_one_epoch(epoch,train_loader)

  running_validation_loss=0.0

  model.eval()

  with torch.no_grad(): # Disable gradient computation and reduce memory consumption
    for i,vdata in enumerate(validation_loader):
      vinputs,vlabels=vdata
      voutputs=model(vinputs)
      vloss=loss_function(voutputs,vlabels)
      running_validation_loss+=vloss
  validation_loss=running_validation_loss/(i+1)
  print('LOSS train: {} validation: {}'.format(train_loss,validation_loss))

  if validation_loss<best_validation_loss: #save the model if it's the best so far
    timestamp=datetime.now().strftime('%Y%m%d_%H%M%S')
    best_validation_loss=validation_loss
    model_path='model_{}_{}'.format(timestamp,epoch)
    torch.save(model.state_dict(),model_path)

#load the best model and evaluate performance on the test set

newModel=simpleCNN()
newModel.load_state_dict(torch.load(model_path))

correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=newModel(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy of the network on the test images: {100*correct/total}%")


EPOCH1:
LOSS train: 2.7235234439373017 validation: 2.735621690750122
EPOCH2:
LOSS train: 2.722459614276886 validation: 2.7345592975616455
EPOCH3:
LOSS train: 2.7216491103172302 validation: 2.734147310256958
EPOCH4:
LOSS train: 2.720754563808441 validation: 2.733184337615967
EPOCH5:
LOSS train: 2.720047777891159 validation: 2.732630968093872
EPOCH6:
LOSS train: 2.719260907173157 validation: 2.7317166328430176
EPOCH7:
LOSS train: 2.7188010573387147 validation: 2.7310402393341064
EPOCH8:
LOSS train: 2.718093979358673 validation: 2.7306413650512695
EPOCH9:
LOSS train: 2.717489355802536 validation: 2.7301743030548096
EPOCH10:
LOSS train: 2.7167446196079252 validation: 2.7295825481414795
EPOCH11:
LOSS train: 2.7162867069244383 validation: 2.729248285293579
EPOCH12:
LOSS train: 2.715808796882629 validation: 2.7286055088043213
EPOCH13:
LOSS train: 2.7151647031307222 validation: 2.728090524673462
EPOCH14:
LOSS train: 2.714818561077118 validation: 2.7278168201446533
EPOCH15:
LOSS train: 2.714281

In [19]:
model = simpleCNN()
#Training
loss_function=nn.CrossEntropyLoss()
#optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
optimizer = optim.Adam(model.parameters())

def train_one_epoch(epoch_index,loader):
  running_loss=0

  for i, data in enumerate(loader):

    inputs,labels=data #get the minibatch
    outputs=model(inputs) #forward pass

    loss=loss_function(outputs,labels) #compute loss
    running_loss+=loss.item() #sum up the loss for the minibatches processed so far

    optimizer.zero_grad() #reset gradients
    loss.backward() #compute gradient
    optimizer.step() #update weights

  return running_loss/(i+1) # average loss per minibatch
EPOCHS=20

best_validation_loss=np.inf

for epoch in range(EPOCHS):
  print('EPOCH{}:'.format(epoch+1))

  model.train(True)
  train_loss=train_one_epoch(epoch,train_loader)

  running_validation_loss=0.0

  model.eval()

  with torch.no_grad(): # Disable gradient computation and reduce memory consumption
    for i,vdata in enumerate(validation_loader):
      vinputs,vlabels=vdata
      voutputs=model(vinputs)
      vloss=loss_function(voutputs,vlabels)
      running_validation_loss+=vloss
  validation_loss=running_validation_loss/(i+1)
  print('LOSS train: {} validation: {}'.format(train_loss,validation_loss))

  if validation_loss<best_validation_loss: #save the model if it's the best so far
    timestamp=datetime.now().strftime('%Y%m%d_%H%M%S')
    best_validation_loss=validation_loss
    model_path='model_{}_{}'.format(timestamp,epoch)
    torch.save(model.state_dict(),model_path)

#load the best model and evaluate performance on the test set

newModel=simpleCNN()
newModel.load_state_dict(torch.load(model_path))

correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=newModel(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy of the network on the test images: {100*correct/total}%")


EPOCH1:
LOSS train: 2.716450572013855 validation: 2.728485107421875
EPOCH2:
LOSS train: 2.696443384885788 validation: 2.69515061378479
EPOCH3:
LOSS train: 2.6649507522583007 validation: 2.683091163635254
EPOCH4:
LOSS train: 2.641265243291855 validation: 2.6622767448425293
EPOCH5:
LOSS train: 2.6185800671577453 validation: 2.661783456802368
EPOCH6:
LOSS train: 2.5942573487758636 validation: 2.6378514766693115
EPOCH7:
LOSS train: 2.5739505171775816 validation: 2.6191976070404053
EPOCH8:
LOSS train: 2.550601136684418 validation: 2.596451759338379
EPOCH9:
LOSS train: 2.5360977828502653 validation: 2.579993963241577
EPOCH10:
LOSS train: 2.5255900263786315 validation: 2.5661733150482178
EPOCH11:
LOSS train: 2.4984161376953127 validation: 2.586270570755005
EPOCH12:
LOSS train: 2.474310564994812 validation: 2.559342861175537
EPOCH13:
LOSS train: 2.452549624443054 validation: 2.5046091079711914
EPOCH14:
LOSS train: 2.434998708963394 validation: 2.4732422828674316
EPOCH15:
LOSS train: 2.42266083

In [20]:
model = simpleCNN()
model.apply(init_weights)
#Training
loss_function=nn.CrossEntropyLoss()
#optimizer=optim.SGD(model.parameters(),lr=0.001,momentum=0.9)
optimizer = optim.Adam(model.parameters())

def train_one_epoch(epoch_index,loader):
  running_loss=0

  for i, data in enumerate(loader):

    inputs,labels=data #get the minibatch
    outputs=model(inputs) #forward pass

    loss=loss_function(outputs,labels) #compute loss
    running_loss+=loss.item() #sum up the loss for the minibatches processed so far

    optimizer.zero_grad() #reset gradients
    loss.backward() #compute gradient
    optimizer.step() #update weights

  return running_loss/(i+1) # average loss per minibatch
EPOCHS=20

best_validation_loss=np.inf

for epoch in range(EPOCHS):
  print('EPOCH{}:'.format(epoch+1))

  model.train(True)
  train_loss=train_one_epoch(epoch,train_loader)

  running_validation_loss=0.0

  model.eval()

  with torch.no_grad(): # Disable gradient computation and reduce memory consumption
    for i,vdata in enumerate(validation_loader):
      vinputs,vlabels=vdata
      voutputs=model(vinputs)
      vloss=loss_function(voutputs,vlabels)
      running_validation_loss+=vloss
  validation_loss=running_validation_loss/(i+1)
  print('LOSS train: {} validation: {}'.format(train_loss,validation_loss))

  if validation_loss<best_validation_loss: #save the model if it's the best so far
    timestamp=datetime.now().strftime('%Y%m%d_%H%M%S')
    best_validation_loss=validation_loss
    model_path='model_{}_{}'.format(timestamp,epoch)
    torch.save(model.state_dict(),model_path)

#load the best model and evaluate performance on the test set

newModel=simpleCNN()
newModel.load_state_dict(torch.load(model_path))

correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=newModel(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy of the network on the test images: {100*correct/total}%")


EPOCH1:
LOSS train: 2.7087710380554197 validation: 2.7117984294891357
EPOCH2:
LOSS train: 2.708067572116852 validation: 2.712247610092163
EPOCH3:
LOSS train: 2.7079052090644837 validation: 2.713569402694702
EPOCH4:
LOSS train: 2.7079037368297576 validation: 2.7133729457855225
EPOCH5:
LOSS train: 2.707606142759323 validation: 2.7152249813079834
EPOCH6:
LOSS train: 2.7075452148914336 validation: 2.716146469116211
EPOCH7:
LOSS train: 2.707583171129227 validation: 2.716646194458008
EPOCH8:
LOSS train: 2.7074816286563874 validation: 2.7176573276519775
EPOCH9:
LOSS train: 2.707358592748642 validation: 2.717883825302124
EPOCH10:
LOSS train: 2.7072682619094848 validation: 2.718186378479004
EPOCH11:
LOSS train: 2.707331484556198 validation: 2.7185616493225098
EPOCH12:
LOSS train: 2.7075271725654604 validation: 2.719860792160034
EPOCH13:
LOSS train: 2.707292765378952 validation: 2.719470739364624
EPOCH14:
LOSS train: 2.707279074192047 validation: 2.719395875930786
EPOCH15:
LOSS train: 2.70747182

In [21]:
"""
model = simpleCNN()
model.apply(init_weights)

loss_function = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters())

def train_one_epoch(epoch_index, loader):
  running_loss = 0

  for i, data in enumerate(loader):

    inputs, labels = data #get the minibatch
    outputs = model(inputs) #forward pass

    loss = loss_function(outputs, labels) #compute loss
    running_loss += loss.item() #sum up the loss for the minibatches processed so far

    optimizer.zero_grad() #reset gradients
    loss.backward() #compute gradient
    optimizer.step() #update weights

  return running_loss / (i + 1) # average loss per minibatch

EPOCHS = 20

best_validation_loss = np.inf

train_losses = []
validation_losses = []
validation_accuracies = []

for epoch in range(EPOCHS):
    print('EPOCH{:>2d}'.format(epoch + 1), end='    ')

    model.train()
    train_loss = train_one_epoch(epoch, train_loader)

    running_validation_loss = 0.0

    model.eval()

    with torch.no_grad():
        total_correct = 0
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_function(voutputs, vlabels)
            running_validation_loss += vloss.item()

            total_correct += (voutputs.argmax(dim=1) == vlabels).sum()
    validation_loss = running_validation_loss / (i + 1)
    validation_acc = total_correct / len(validation_loader.dataset) * 100
    print('LOSS train: {:1.3f} validation: {:1.3f} | ACC val: {:>5.1f}%'.format(
        train_loss, validation_loss, validation_acc
    ))

    if validation_loss < best_validation_loss: #save the model if it's the best so far
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        best_validation_loss = validation_loss
        model_path = 'model_{}_{}'.format(timestamp, epoch)
        torch.save(model.state_dict(), model_path)

    train_losses.append(train_loss)
    validation_losses.append(validation_loss)
    validation_accuracies.append(validation_acc)

import matplotlib.pyplot as plt
plt.plot(train_losses, color='tab:red', linewidth=3, label='train loss')
plt.plot(validation_losses, color='tab:green', linewidth=3, label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('CE loss')

ax_right = plt.gca().twinx()
ax_right.plot(validation_accuracies, color='tab:green', linestyle='--', label='validation accuracy')
ax_right.set_ylabel('accuracy (%)')

plt.gcf().legend(ncol=3)
plt.gcf().set_size_inches(6, 3)
"""

"\nmodel = simpleCNN()\nmodel.apply(init_weights)\n\nloss_function = nn.CrossEntropyLoss()\n#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)\noptimizer = optim.Adam(model.parameters())\n\ndef train_one_epoch(epoch_index, loader):\n  running_loss = 0\n\n  for i, data in enumerate(loader):\n\n    inputs, labels = data #get the minibatch\n    outputs = model(inputs) #forward pass\n\n    loss = loss_function(outputs, labels) #compute loss\n    running_loss += loss.item() #sum up the loss for the minibatches processed so far\n\n    optimizer.zero_grad() #reset gradients\n    loss.backward() #compute gradient\n    optimizer.step() #update weights\n\n  return running_loss / (i + 1) # average loss per minibatch\n\nEPOCHS = 20\n\nbest_validation_loss = np.inf\n\ntrain_losses = []\nvalidation_losses = []\nvalidation_accuracies = []\n\nfor epoch in range(EPOCHS):\n    print('EPOCH{:>2d}'.format(epoch + 1), end='    ')\n\n    model.train()\n    train_loss = train_one_epoch(epoch,

In [22]:
#load the best model and evaluate performance on the test set

newModel=simpleCNN()
newModel.load_state_dict(torch.load(model_path))

correct=0
total=0

with torch.no_grad():
  for data in test_loader:
    images,labels=data
    outputs=newModel(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

print(f"Accuracy of the network on the test images: {100*correct/total}%")


Accuracy of the network on the test images: 7.202680067001675%
